# Reddit Scraping
In order to scrape posts from Reddit, we used the [`praw` python library](https://github.com/praw-dev/praw). 
In order to use that, we need:
- a Reddit account.
- a Reddit app ([here](https://www.reddit.com/prefs/apps) you can create one). Here you'll find the client id under the app name once you create it, as well as the client secret and the user agent, which is your app name.

Once we have those, we can access reddit from the terminal using these steps(it is also possible to access as a user by adding username and account if you want to use it in write mode):

In [3]:
#pip install praw
import praw
reddit = praw.Reddit(client_id='', client_secret='', user_agent='')

We can analyse posts, post requirements etc. for a single subreddit or for all of them(by using the subreddit 'all'); we can also select top posts or do searches. 
# sistemare data utc

In [2]:
for i in reddit.subreddit('all').top(limit=5): #this will print the top posts all time
    print(i.title + ' Author: ' + i.author.name +  ' Link: https://www.reddit.com' + i.permalink + ' Subreddit: r/' + i.subreddit.display_name + ' Upvote Ratio: ' + str(i.upvote_ratio) + ' Date: '+str(i.created_utc))


I’ve found a few funny memories during lockdown. This is from my 1st tour in 89, backstage in Vegas. Author: ReallyRickAstley Link: https://www.reddit.com/r/pics/comments/haucpf/ive_found_a_few_funny_memories_during_lockdown/ Subreddit: r/pics Upvote Ratio: 0.99 Date: 1592410647.0
Times Square right now Author: SomeGuyInDeutschland Link: https://www.reddit.com/r/wallstreetbets/comments/l8rf4k/times_square_right_now/ Subreddit: r/wallstreetbets Upvote Ratio: 0.99 Date: 1612029638.0
Joe Biden elected president of the United States Author: throwawaynumber53 Link: https://www.reddit.com/r/news/comments/jptqj9/joe_biden_elected_president_of_the_united_states/ Subreddit: r/news Upvote Ratio: 0.88 Date: 1604766517.0
The Senate. Upvote this so that people see it when they Google "The Senate". Author: serventofgaben Link: https://www.reddit.com/r/movies/comments/62sjuh/the_senate_upvote_this_so_that_people_see_it_when/ Subreddit: r/movies Upvote Ratio: 0.96 Date: 1491051474.0
My cab driver toni

# Inserire spiegazione differenza tra le diverse scale hot top controversial
Now, for our analysis we will use data from the subreddit r/conspiracy and we will try to study the ramifications of the posts.

In [7]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

#put the subreddit already passed here
id_to_avoid = ["conspiracy"]

for i in reddit.subreddit("conspiracy").hot(limit=100):
    post_list.append((i.title, i.score, "https://www.reddit.com"+i.permalink))


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]):
        if repost.subreddit_id not in id_to_avoid:
            subreddit_url = str(repost.subreddit)
            subreddit_url = "https://www.reddit.com/r/" + subreddit_url
            if subreddit_url in conspiracy_dict.keys():
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1] +=1
            else:
                conspiracy_dict[subreddit_url]=[[],[]]
                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                conspiracy_dict[subreddit_url][1] =1


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_hot.csv',index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'result/conspiracy_hot.csv'

An interesting thing we noted is that a good part of the propagation of news comes from bots, especially from the top posts.